In [1]:
from typing import Tuple
import pandas as pd
from tqdm.notebook import tqdm
from evaluation.generated_dataset import GeneratedDataset, load_all_from_config
from evaluation.novelty import NoveltyFilter, filter_by_unique_structure

In [2]:
datasets = {
    #"WyckoffTransformer-raw": ("WyckoffTransformer",),
    "WyckoffTransformer-CrySPR": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyckoffTransformer-DiffCSP++": ("WyckoffTransformer", "DiffCSP++", "CHGNet_fix"),
    #"WyckoffTransformer-free": ("WyckoffTransformer", "CrySPR", "CHGNet_free"),
    "CrystalFormer": ("CrystalFormer", "CHGNet_fix_release"),
    #"DiffCSP++ raw": ("DiffCSP++",),
    "DiffCSP++": ("DiffCSP++", "CHGNet_fix_release"),
    "DiffCSP": ("DiffCSP", "CHGNet_fix"),
    "FlowMM": ("FlowMM-relaxed",),
    #"MP-20 train": ("split", "train"),
    #"MP-20 test": ("split", "test"),
}

In [3]:
all_datasets = load_all_from_config()

In [4]:
novelty_reference = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
novelty_filter = NoveltyFilter(novelty_reference)

In [5]:
import evaluation.statistical_evaluator
#import importlib
#importlib.reload(evaluation.statistical_evaluator)
test_unique = filter_by_unique_structure(all_datasets[('split', 'test')].data)
test_novel = novelty_filter.get_novel(test_unique)
test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(test_novel)

In [6]:
import evaluation.novelty
import importlib
importlib.reload(evaluation.novelty)
train_w_template_set = frozenset(novelty_reference.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1))

In [7]:
def is_sg_preserved(relaxed_sg, transformations: Tuple[str]) -> pd.Series:
    source_sg = all_datasets[(transformations[0],)].data.spacegroup_number
    return relaxed_sg == source_sg.reindex_like(relaxed_sg)

In [8]:
mp_20 = pd.concat([
    all_datasets[('split', 'train')].data,
    all_datasets[('split', 'test')].data,
    all_datasets[('split', 'val')].data], axis=0, verify_integrity=True)
(mp_20.spacegroup_number == 1).mean()
mp_20.smact_validity.mean()

0.9057020937893829

In [9]:
1-mp_20.smact_validity.mean()

0.09429790621061707

In [85]:
len(unique.loc[novel_template])

171

In [88]:
table = pd.DataFrame(
    index=datasets.keys(), columns=[
        "Novelty (%)", "Structural", "Compositional", 
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements",
        "S.U.N. (%)",
        "Novel Template (%)", "P1 (%)",
        "Space Group", "S.S.U.N. (%)"])
table.index.name = "Method"
E_hull_threshold = 0.08
unique_sample_size = 992
for name, transformations in tqdm(datasets.items()):
    dataset = all_datasets[transformations]
    unique = filter_by_unique_structure(dataset.data)
    novel_template = ~unique.apply(evaluation.novelty.record_to_anonymous_fingerprint, axis=1).isin(train_w_template_set)
    table.loc[name, "Novel Template (%)"] = 100 * novel_template.mean()
    if transformations == ("split", "train"):
        novel = unique
    else:
        novel = novelty_filter.get_novel(unique)
    table.loc[name, "Novelty (%)"] = 100 * len(novel) / len(unique)
    if "structural_validity" in novel.columns:
        table.loc[name, "Structural"] = 100 * novel.structural_validity.mean()
        table.loc[name, "Compositional"] = 100 * novel.smact_validity.mean()
    if "cdvae_crystal" in novel.columns:
        cov_metrics = test_evaluator.get_coverage(novel.cdvae_crystal)    
        table.loc[name, "Recall"] = 100 * cov_metrics["cov_recall"]
        table.loc[name, "Precision"] = 100 * cov_metrics["cov_precision"]
        novel = novel[novel.structural_validity]
        table.loc[name, r"$\rho$"] = test_evaluator.get_density_emd(novel)
        table.loc[name, "$E$"] = test_evaluator.get_cdvae_e_emd(novel)
        table.loc[name, "# Elements"] = test_evaluator.get_num_elements_emd(novel)
    table.loc[name, "P1 (%)"] = 100 * (novel.group == 1).mean()
    # table.loc[name, "# DoF"] = test_evaluator.get_dof_emd(novel)
    table.loc[name, "Space Group"] = test_evaluator.get_sg_chi2(novel)
    #try:
    #    table.loc[name, "SG preserved (%)"] = 100 * is_sg_preserved(novel.spacegroup_number, transformations).mean()
    #except KeyError:
    #    pass
    #table.loc[name, "Elements"] = test_evaluator.get_elements_chi2(novel)
    if "corrected_chgnet_ehull" in novel.columns:
        # S.U.N. is measured with respect to the initial structures
        has_ehull = dataset.data.corrected_chgnet_ehull.notna().sum()
        is_sun = (novel.corrected_chgnet_ehull <= E_hull_threshold)
        table.loc[name, "S.U.N. (%)"] = 100 * is_sun.sum() / has_ehull
        table.loc[name, "S.S.U.N. (%)"] = 100 * (is_sun & (novel.group != 1)).sum() / has_ehull
    if transformations == ("split", "train"):
        # Train forms the baseline of the hull
        test_dataset = all_datasets[("split", "test", "CHGNet_fix")].data
        test_with_ehull = test_dataset[test_dataset.corrected_chgnet_ehull.notna()]
        test_unique = filter_by_unique_structure(test_with_ehull)
        test_novel = novelty_filter.get_novel(test_unique)
        table.loc[name, "S.U.N. (%)"] = 100 * (test_novel.corrected_chgnet_ehull <= E_hull_threshold).sum() / len(test_with_ehull)
table

  0%|          | 0/7 [00:00<?, ?it/s]

900 198 d
895 200 d
760 96 d
896 10 d
897 95 d
8156 820 d
523 171 d


,Novelty (%),Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements,S.U.N. (%),Novel Template (%),P1 (%),Space Group,S.S.U.N. (%)
Method,,,,,,,,,,,,,
WyckoffTransformer-CrySPR,90.0,99.555556,80.444444,98.666667,96.715328,0.73714,0.053159,0.096566,38.938939,19.8,3.236607,0.222677,38.038038
WyckoffTransformer-DiffCSP++,89.5,99.664804,80.335196,99.217877,96.792745,0.665291,0.050328,0.098188,36.6,20.0,1.457399,0.211951,35.9
CrystalFormer,76.923077,86.842105,82.368421,99.868421,95.13382,0.524171,0.099555,0.163017,33.870968,9.716599,0.909091,0.276161,33.770161
DiffCSP++,89.68969,100.0,85.044643,99.330357,95.79739,0.147837,0.036122,0.503632,41.4,1.001001,2.566964,0.25525,40.8
DiffCSP,90.060241,100.0,80.936455,99.554069,96.206591,0.822378,0.052409,0.293636,57.4,9.538153,36.566332,7.988511,40.6
FlowMM,89.439632,100.0,81.927415,99.668955,99.635036,0.489733,0.035512,0.130643,NaN,8.992214,44.274154,12.423176,NaN
WyCryst,52.615694,99.808795,75.525813,98.852772,87.104623,0.962397,0.112944,0.285756,36.619718,17.203219,4.789272,0.710472,35.211268


In [11]:
table.to_csv("tables/paper_summary_table.csv")
table.to_pickle("tables/paper_summary_table.pkl")

In [12]:
max_subset=["Novelty (%)", "Structural", "Compositional", "Recall", "Precision", "S.S.U.N. (%)", "S.U.N. (%)", "Novel Template (%)"]
# -1 to exclude the MP-20 training set
def highlight_max_value(s):
    if s.name not in max_subset:
        return ['' for _ in s]
    is_max = s == s.max()
    #is_max.iloc[-1] = False
    return ['font-weight: bold' if v else '' for v in is_max]

min_subset=[r"$\rho$", "$E$", "# Elements", "# DoF", "Space Group", "Elements", "P1 (%)"]
def highlight_min_value(s):
    if s.name not in min_subset:
        return ['' for _ in s]
    is_min = s == s.min()
    #is_min.iloc[-1] = False
    return ['font-weight: bold' if v else '' for v in is_min]

In [13]:
def prettify(table):
    return table.style.format({
    "Novelty (%)": "{:.2f}",
    "Structural": "{:.2f}",
    "Compositional": "{:.2f}",
    "Recall": "{:.2f}",
    "Precision": "{:.2f}",
    r"$\rho$": "{:.2f}",
    "$E$": "{:.3f}",
    "# Elements": "{:.3f}",
    "# DoF": "{:.3f}",
    "Space Group": "{:.3f}",
    "Elements": "{:.3f}",
    "Novel Template (%)": "{:.2f}",
    "P1 (%)": "{:.2f}",
    "S.U.N. (%)": "{:.1f}",
    "S.S.U.N. (%)": "{:.1f}",
}).apply(highlight_max_value).apply(highlight_min_value)
prettify(table)

,Novelty (%),Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements,S.U.N. (%),Novel Template (%),P1 (%),Space Group,S.S.U.N. (%)
Method,,,,,,,,,,,,,
WyckoffTransformer-CrySPR,90.00,99.56,80.44,98.67,96.72,0.74,0.053,0.097,38.9,19.80,3.24,0.223,38.0
WyckoffTransformer-DiffCSP++,89.50,99.66,80.34,99.22,96.79,0.67,0.050,0.098,36.6,20.00,1.46,0.212,35.9
CrystalFormer,76.92,86.84,82.37,99.87,95.13,0.52,0.100,0.163,33.9,9.72,0.91,0.276,33.8
DiffCSP++,89.69,100.00,85.04,99.33,95.80,0.15,0.036,0.504,41.4,1.00,2.57,0.255,40.8
DiffCSP,90.06,100.00,80.94,99.55,96.21,0.82,0.052,0.294,57.4,9.54,36.57,7.989,40.6
FlowMM,89.44,100.00,81.93,99.67,99.64,0.49,0.036,0.131,nan,8.99,44.27,12.423,nan


In [14]:
prettify(table.iloc[:, :9]).to_latex("tables/summary_similarity_raw.tex", siunitx=True, convert_css=True)
prettify(table.iloc[:, 9:]).to_latex("tables/summary_symmetry_raw.tex", siunitx=True, convert_css=True)

In [15]:
datasets["WyCryst"] = ('WyCryst', 'CHGNet_fix')
all_datasets[('WyCryst', 'CHGNet_fix')] = GeneratedDataset.from_cache(('WyCryst', 'CHGNet_fix'), "mp_20_biternary")

In [16]:
raw_test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)
cdvae_table = pd.DataFrame(index=pd.Index(datasets.keys(), tupleize_cols=False),
    columns=[
        "Structural", "Compositional",
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements"])
sample_size = 992
for name, transformations in tqdm(datasets.items()):
    dataset = all_datasets[transformations]
    if "structure" in dataset.data.columns:
        cdvae_table.loc[name, "Compositional"] = 100*dataset.data.smact_validity.mean()
        cdvae_table.loc[name, "Structural"] = 100*dataset.data.structural_validity.mean()
        valid = dataset.data[dataset.data.naive_validity]
        cov_metrics = raw_test_evaluator.get_coverage(valid.cdvae_crystal)
        cdvae_table.loc[name, "Recall"] = 100*cov_metrics["cov_recall"]
        cdvae_table.loc[name, "Precision"] = 100*cov_metrics["cov_precision"]
        cdvae_table.loc[name, r"$\rho$"] = raw_test_evaluator.get_density_emd(valid)
        cdvae_table.loc[name, "$E$"] = raw_test_evaluator.get_cdvae_e_emd(valid)
        cdvae_table.loc[name, "# Elements"] = raw_test_evaluator.get_num_elements_emd(valid)
cdvae_table.to_csv("tables/cdvae_metrics_table.csv")
prettify(cdvae_table).to_latex("tables/cdvae_metrics_table.tex", siunitx=True, convert_css=True)
prettify(cdvae_table)

  0%|          | 0/7 [00:00<?, ?it/s]

,Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements
WyckoffTransformer-CrySPR,99.60,81.40,98.77,95.94,0.39,0.078,0.081
WyckoffTransformer-DiffCSP++,99.70,81.40,99.26,95.85,0.33,0.070,0.078
CrystalFormer,89.92,84.88,99.87,95.45,0.19,0.139,0.119
DiffCSP++,100.00,85.80,99.42,95.48,0.13,0.036,0.453
DiffCSP,100.00,82.50,99.64,95.18,0.46,0.075,0.321
FlowMM,100.00,82.83,99.71,99.56,0.17,0.046,0.093
WyCryst,99.90,82.09,99.63,96.16,0.44,0.330,0.322


In [74]:
raw_datasets = {
    "WyckoffTransformer": ("WyckoffTransformer", "CrySPR", "CHGNet_fix_release"),
    "WyFormerDiffCSP++": ("WyckoffTransformer", "DiffCSP++"),
    "CrystalFormer": ("CrystalFormer",),
    "DiffCSP++": ("DiffCSP++",),
    "WyCryst": ('WyCryst', 'CHGNet_fix'),
    "DiffCSP": ("DiffCSP",),
    "FlowMM": ("FlowMM",),
}
raw_test_evaluator = evaluation.statistical_evaluator.StatisticalEvaluator(all_datasets[('split', 'test')].data)
cdvae_table = pd.DataFrame(index=pd.Index(raw_datasets.keys(), tupleize_cols=False),
    columns=[
        "Structural", "Compositional",
        "Recall", "Precision",
        r"$\rho$", "$E$", "# Elements"])
sample_size = 992
for name, transformations in tqdm(raw_datasets.items()):
    dataset = all_datasets[transformations]
    if "structure" in dataset.data.columns:
        cdvae_table.loc[name, "Compositional"] = 100*dataset.data.smact_validity.mean()
        cdvae_table.loc[name, "Structural"] = 100*dataset.data.structural_validity.mean()
        valid = dataset.data[dataset.data.naive_validity]
        cov_metrics = raw_test_evaluator.get_coverage(valid.cdvae_crystal)
        cdvae_table.loc[name, "Recall"] = 100*cov_metrics["cov_recall"]
        cdvae_table.loc[name, "Precision"] = 100*cov_metrics["cov_precision"]
        cdvae_table.loc[name, r"$\rho$"] = raw_test_evaluator.get_density_emd(valid)
        cdvae_table.loc[name, "$E$"] = raw_test_evaluator.get_cdvae_e_emd(valid)
        cdvae_table.loc[name, "# Elements"] = raw_test_evaluator.get_num_elements_emd(valid)
cdvae_table.to_csv("tables/cdvae_metrics_no_relax_table.csv")
prettify(cdvae_table).to_latex("tables/cdvae_metrics_no_relax_table.tex", siunitx=True, convert_css=True)
prettify(cdvae_table)

  0%|          | 0/7 [00:00<?, ?it/s]

,Structural,Compositional,Recall,Precision,$\rho$,$E$,# Elements
WyckoffTransformer,99.60,81.40,98.77,95.94,0.39,0.078,0.081
WyFormerDiffCSP++,99.80,81.40,99.51,95.81,0.36,0.083,0.079
CrystalFormer,93.39,84.98,99.62,94.56,0.19,0.208,0.128
DiffCSP++,99.94,85.13,99.67,99.54,0.31,0.069,0.399
WyCryst,99.90,82.09,99.63,96.16,0.44,0.330,0.322
DiffCSP,100.00,83.20,99.82,99.51,0.35,0.095,0.347
FlowMM,96.87,83.11,99.73,99.39,0.12,0.073,0.094


In [18]:
from scipy.stats import wasserstein_distance
import numpy as np

In [70]:
a = np.random.normal(0.1, 1, 1000)
b = np.random.normal(0, 1, 1000)

In [71]:
wasserstein_distance(a, b)

0.09797184802871513